# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [86]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [87]:
# enter your code here
#orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders = pd.read_csv("/Users/admin/Desktop/Ironhack/02-Codes/Notebooks/notebooks/data/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [88]:
orders.index

RangeIndex(start=0, stop=397924, step=1)

In [89]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [90]:
# your code here
# Sub Problem 1
# We'll work only with the fields that are relevant for our analysis. 
orders = pd.pivot_table(orders, index=['CustomerID', 'Country'], values='amount_spent', aggfunc=np.sum, fill_value=0)
orders = orders.reset_index()
orders.head()

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40


In [91]:
#Sub Problem 2
vip_perc = round(np.quantile(orders.groupby('CustomerID')['amount_spent'].sum(), 0.95), 2)
print(f'The 95th percentile for aggregated expenses is {vip_perc}')

The 95th percentile for aggregated expenses is 5840.18


In [92]:
preferred_perc = round(np.quantile(orders.groupby('CustomerID')['amount_spent'].sum(), 0.75), 2)
print(f'The 75th percentile for aggregated expenses is {preferred_perc}')

The 75th percentile for aggregated expenses is 1661.64


In [93]:
# Sub Problem 3
def CustomerType(x):
    if x['amount_spent'] >= vip_perc:
        return 'VIP'
    elif x['amount_spent'] >= preferred_perc:
        return 'Preferred'
    else:
        return 'Regular'
    
orders['CustomerType'] = orders.apply(CustomerType, axis=1)
orders.head()

,CustomerID,Country,amount_spent,CustomerType
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [94]:
# your code here
country_1 = orders[orders['CustomerType'] == 'VIP'].groupby('Country').count()
country_1.nlargest(1,'CustomerType')

,CustomerID,amount_spent,CustomerType
Country,,,
United Kingdom,177,177,177


In [95]:
country_2 = orders[orders['CustomerType'] != 'Regular'].groupby('Country').count()
country_2.nlargest(1,'CustomerType')

,CustomerID,amount_spent,CustomerType
Country,,,
United Kingdom,932,932,932
